In [ ]:
from ngsolve import *

from ngsolve.meshes import *

from ngsolve.solvers import *
from ngsolve.webgui import Draw

import matplotlib.pyplot as plt

from su2_yangmills import *

import numpy as np

sys.path.insert(1, '../quaternion/')

from quaternion_cf import *

In [ ]:
ne=40
mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
#mesh = MakeStructured2DMesh(quads=False, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
#Draw(mesh)

In [ ]:
#fesC = H1(mesh)**3 # for su2 charge
#fesA = HCurl(mesh)**3 # for su2 gauge potential
order=4
fesC = Periodic(H1(mesh, order=order))**3 # for su2 charge
fesA = Periodic(HCurl(mesh, order=order-1))**3 # for su2 gauge potential
#fesA = Periodic(VectorH1(mesh, order=order-1))**3 # for su2 gauge potential
fesAz = Periodic(H1(mesh, order=order))**3 # for su2 gauge potential z component
fesA3D = fesA*fesAz

fesA3DC = fesA3D*fesC

In [ ]:
#phi1_CF = CF( gaussxyper([0,0], 1.) )
phi1_CF = CF( sin(4*pi*x) )

#n1_CF = CF( (0,0,1) )
#n1_CF = CF( (0,1./sqrt(2),1./sqrt(2)) )
#n1_CF = CF( (1./sqrt(2),0,1./sqrt(2)) )
n1_CF = CF( (1./sqrt(2),1./sqrt(2),0) )

#phi2_CF = CF( 0 )
#phi2_CF = -1.*phi1_CF
phi2_CF = phi1_CF
n2_CF = n1_CF

q1 = get_rotqCF(phi1_CF, n1_CF)
q2 = get_rotqCF(phi2_CF, n2_CF)

#q1q2 = qmul(q1,q2)
q1q2 = qmulCF(q1,qconjCF(q2))
#q1q2 = qmul(q2,q1)


#print(q1q2.dims)

#Remark: the norm in Lie Algebra space should be invariant under gauges!
#q_dict = {"q1q2:": (q1q2, {})}

q_dict = {}

#for i in range(4):
#    q_dict[f"q1q2[{i}]"] = (q1q2[i], {})

generate_scenes(q_dict, mesh=mesh)

In [ ]:
q1 = CF( (0,1,0,0) ).Reshape( (4,) )

su2mat1 = su2mat_from_q(q1)
print(su2mat1.dims)

su2_dict = {}

for i in range(2):
    for j in range(2):
        su2_dict[f"su2mat1[{i},{j}]"] = (su2mat1[i,j], {})

generate_scenes(su2_dict, mesh=mesh)

In [ ]:
q1 = CF( (0,1,0,0) ).Reshape( (4,) )

su2mat1 = su2mat_from_q(q1)


#q1q1 = qmul(q1,q1)
su2mat12 = matmul(su2mat1,su2mat1)

draw_dict = {}

#draw_dict[f"00"] = (su2mat1[0,0]*su2mat1[0,0]+su2mat1[0,1]*su2mat1[1,0], {})
#draw_dict[f"00"] = (InnerProduct(su2mat1[0,:], su2mat1[:,0]), {})

for i in range(2):
    for j in range(2):
        draw_dict[f"su2mat12[{i},{j}]"] = (su2mat12[i,j], {})
        
#for i in range(4):
#    draw_dict[f"q1q1[{i}]"] = (q1q1[i], {})

generate_scenes(draw_dict, mesh=mesh)

In [ ]:
q1 = CF( (0,1,0,0) ).Reshape( (4,) )
q2 = CF( (0,0,1,0) ).Reshape( (4,) )

su2mat1 = su2mat_from_q(q1)
su2mat2 = su2mat_from_q(q2)
su2mat12 = matmul(su2mat1,su2mat2) - matmul(su2mat2,su2mat1)


draw_dict = {}

#draw_dict[f"00"] = (su2mat1[0,0]*su2mat1[0,0]+su2mat1[0,1]*su2mat1[1,0], {})
#draw_dict[f"00"] = (InnerProduct(su2mat1[0,:], su2mat1[:,0]), {})

for i in range(2):
    for j in range(2):
        draw_dict[f"su2mat12[{i},{j}]"] = (su2mat12[i,j], {})
        
#for i in range(4):
#    draw_dict[f"q1q1[{i}]"] = (q1q1[i], {})

generate_scenes(draw_dict, mesh=mesh)

In [ ]:
q1 = CF( (0,1,0,0) ).Reshape( (4,) )
q2 = CF( (0,0,1,0) ).Reshape( (4,) )

def compare_qmul_matmul(q1,q2):
    
    q1q2 = qmulCF(q1,q2)
    su2mat12_from_q = su2mat_from_q(q1q2)

    su2mat1 = su2mat_from_q(q1)
    su2mat2 = su2mat_from_q(q2)
    su2mat12 = matmul(su2mat1,su2mat2)
    
    return su2mat12_from_q - su2mat12


draw_dict = {}

diff12 = compare_qmul_matmul(q1,q2)

absdiff12 = Integrate(Norm(diff12), mesh)
print(absdiff12)
draw_dict["Norm(diff12)"] = (Norm(diff12), {})

#for i in range(2):
#    for j in range(2):
#        draw_dict[f"diff12[{i},{j}]"] = (diff12[i,j], {})
        
#for i in range(4):
#    draw_dict[f"q1q1[{i}]"] = (q1q1[i], {})

generate_scenes(draw_dict, mesh=mesh)

In [ ]:
def compare_qmul_matmul(q1,q2):
    
    q1q2 = qmulCF(q1,q2)
    su2mat12_from_q = su2mat_from_q(q1q2)

    su2mat1 = su2mat_from_q(q1)
    su2mat2 = su2mat_from_q(q2)
    su2mat12 = matmul(su2mat1,su2mat2)
    
    return su2mat12_from_q - su2mat12



tries = 100

for i in range(tries):
    
    qrands = np.random.rand(2,4)
    
    #print(tuple(qrands[0]))
    #print(tuple(qrands[1]))
    
    q1 = CF( tuple(qrands[0]) ).Reshape( (4,) )
    q2 = CF( tuple(qrands[1]) ).Reshape( (4,) )

    diff12 = compare_qmul_matmul(q1,q2)
    absdiff12 = Integrate(Norm(diff12), mesh)
    print(absdiff12)


In [ ]:
def compare_qmul_matmul(q1,q2):
    
    q1q2 = qmulCF(q1,q2)
    su2mat12_from_q = su2mat_from_q(q1q2)

    su2mat1 = su2mat_from_q(q1)
    su2mat2 = su2mat_from_q(q2)
    su2mat12 = matmul(su2mat1,su2mat2)
    
    return su2mat12_from_q - su2mat12



tries = 100

for i in range(tries):
    
    qrands = np.random.rand(1,4)
    
    #print(tuple(qrands[0]))
    #print(tuple(qrands[1]))
    
    q1 = CF( tuple(qrands[0]) ).Reshape( (4,) )
    q2 = qconjCF(q1)

    diff12 = compare_qmul_matmul(q1,q2)
    absdiff12 = Integrate(Norm(diff12), mesh)
    print(absdiff12)


In [ ]:
def rot(q,qrot):
    
    return qmul(qconj(qrot), qmul(q,qrot))

def rotCF(q,qrot):
    
    return qmulCF(qconjCF(qrot), qmulCF(q,qrot))



tries = 100

#q = np.random.rand(4)
q = CF( tuple(np.random.rand(4)) )


for i in range(tries):
    
    thetarand = np.random.random_sample()*pi
    phirand = np.random.random_sample()*2.*pi
    rotanglerand = np.random.random_sample()*2.*pi
    nrand = axis(thetarand, phirand)
    
    #print(np.linalg.norm(nrand))
    
    #qrand = get_rotq(rotanglerand, nrand)
    qrand = CF( tuple(get_rotq(rotanglerand, nrand)) )
    
    #print(np.linalg.norm(qrand))
    
    #qnorm = np.linalg.norm(q)
    #newqnorm = np.linalg.norm(rot(q, qrand))
    
    qnorm = Norm(q)
    newqnorm = Norm(rotCF(q, qrand))
    
    normdiff = Integrate(qnorm - newqnorm, mesh)
    #print(normdiff)
    if normdiff > 10E-15:
        print("rotation broken")
